In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import cupy as cp
EPSILON=1e-8

In [ ]:
def plot_random_sample_and_recovery(random_sample,recovered_random_sample):
    # Generate indices for plotting
    indices = np.arange(random_sample.shape[0])
    frequencies = np.fft.fftfreq(random_sample.shape[-1])

    # Create a dataframe for the original and recovered random samples for easy plotting with Plotly
    df_samples = pd.DataFrame({
        'Index': np.concatenate([indices, indices]),
        'Value': np.concatenate([random_sample, recovered_random_sample.real]),
        'Sample Type': ['Original' for _ in range(len(random_sample))] + ['Recovered' for _ in range(len(recovered_random_sample))]
    })

    # Plot original and recovered random sample using Plotly Express
    fig_samples = px.line(df_samples, x='Index', y='Value', color='Sample Type', title='Original vs. Recovered Random Sample')
    return fig_samples

In [ ]:
def plot_frequencies(random_sample,fft_result_random):
    # Create a dataframe for the FFT magnitude plot for easy plotting with Plotly
    frequencies=np.fft.fftfreq(random_sample.shape[-1])
    df_fft = pd.DataFrame({
        'Frequency': frequencies,
        'Magnitude': np.abs(fft_result_random),
        'Phase': np.angle(fft_result_random),
    })

    # Plot FFT magnitude of random sample using Plotly Express
    fig_fft = px.scatter(df_fft, x='Frequency', y='Magnitude', color='Phase',title='FFT Magnitude of Random Sample')
    return fig_fft

In [ ]:
# Generate a random sample
np.random.seed(42)  # For reproducibility
sample_size=1024
random_sample = np.random.randn(sample_size)  # Generate 1024 random numbers from a normal distribution

In [ ]:
# Perform FFT on the random sample
fft_result_random = np.fft.fft(random_sample)
# Perform IFFT to recover the original random sample
recovered_random_sample = np.fft.ifft(fft_result_random)

In [ ]:
fft_result_random

In [ ]:
plot_random_sample_and_recovery(random_sample,recovered_random_sample)

In [ ]:
plot_frequencies(random_sample,fft_result_random)

In [ ]:
# --------------------------------------------------------------------
#  ⚡️ Readability–first reference implementations of the 1‑D DFT
#  ---------------------------------------------------------------
#  These are intentionally *not* the fastest way to compute an FFT
#  (use np.fft or cupy.fft in production).  They are written to
#  mirror the mathematics as closely as possible, so every line has
#  an obvious theoretical counterpart.
# --------------------------------------------------------------------

def my_fft(x: np.ndarray):
    """
    Discrete Fourier Transform (DFT) — reference implementation.

    Parameters
    ----------
    x : ndarray, shape (N,)
        Time‑domain signal.  May be real or complex.

    Returns
    -------
    X : ndarray, shape (N,)
        Frequency‑domain spectrum where element `k` equals

        .. math::

            X[k] = \\sum_{n=0}^{N-1} x[n] \\; e^{-j\\,2\\pi kn / N}.

    Notes
    -----
    * **Why the outer‑product form?**  
      We build the full `N×N` matrix

      .. math:: e^{-j 2\\pi k n /N}

      (rows indexed by *k*, columns by *n*) and perform an ordinary
      matrix–vector multiplication.  That matches the textbook
      definition exactly, at the cost of 𝒪(N²) flops and memory.

    * **Orthogonality**  
      The complex exponentials for different *k* are mutually
      orthogonal under the discrete inner product, which is why the
      transform is invertible.

    * **Performance**  
      A radix‑2 FFT reduces the cost to 𝒪(N log N).  This routine is
      therefore suitable for pedagogy or unit‑testing only.
    """
    N  = len(x)
    n  = np.arange(N)                 # 0 … N‑1  (time indices)
    k  = n.reshape((N, 1))            # 0 … N‑1  (frequency indices as column)
    e  = np.exp(-2j * np.pi * k * n / N)
    return e @ x                      # same as np.dot(e, x)


def my_ifft(X: np.ndarray):
    """
    Inverse Discrete Fourier Transform (IDFT) — reference implementation.

    Parameters
    ----------
    X : ndarray, shape (N,)
        Complex Fourier spectrum produced by `my_fft` (or any routine
        following the same conventions).

    Returns
    -------
    x : ndarray, shape (N,)
        Reconstructed time‑domain sequence:

        .. math::

            x[n] = \\frac{1}{N} \\sum_{k=0}^{N-1} X[k] \\; e^{+j\\,2\\pi kn / N}.

    Notes
    -----
    * The only difference from `my_fft` is the sign in the exponent
      and the **1/N scaling** (required to make the transform pair
      unitary).  Many FFT packages put the 1/N factor in the *forward*
      transform or split it as 1/√N in both directions; any choice is
      valid as long as the pair is mutually inverse.

    * Because the algorithm is again 𝒪(N²), use it for educational
      purposes, small N, or validation of high‑performance FFT code.
    """
    N  = len(X)
    n  = np.arange(N)
    k  = n.reshape((N, 1))
    e  = np.exp(+2j * np.pi * k * n / N)
    return (e @ X) / N                # divide by N to undo the forward scale


def fftfreq(N: int, d: float = 1.0):
    r"""
    Frequency bin centres corresponding to an *N‑point* FFT.

    The FFT output `X[0 … N−1]` is indexed by an integer *k*.  The
    physical frequency represented by that bin is

    .. math:: f_k = \frac{k}{N\,d} \quad\text{for}\; k = 0,1,…,N-1.

    Because complex exponentials are periodic, indices beyond `N/2`
    actually correspond to *negative* frequencies.  This helper
    returns them in the exact order stored by `numpy.fft.fft`, i.e.

        ``[0, +f₀, +2f₀, …, +(N/2−1)f₀,  −N/2 f₀, …, −f₀]``

    where ``f₀ = 1 / (N d)``.

    Parameters
    ----------
    N : int
        Length of the FFT (window size).

    d : float, optional
        Sample spacing in **time** units.  If the data were collected
        at sampling frequency ``fs`` Hz, set ``d = 1/fs``.  Default is
        1.0, which yields dimensionless “cycles per sample”.

    Returns
    -------
    f : ndarray, shape (N,)
        Frequencies in Hertz (or `1/d` units) that align with the FFT
        bins.

    Implementation details
    ----------------------
    Using modular arithmetic we can jump straight to the desired
    ordering without splicing arrays:

    1. ``k = [0, 1, …, N−1]``
    2. ``k + N//2``         ⇒ shift origin by half a window
    3. ``% N``              ⇒ wrap values into `[0, …, N−1]`
    4. ``− N//2``           ⇒ recentre so zero is in the middle
    5. divide by ``N*d``    ⇒ convert index → frequency

    This matches `numpy.fft.fftfreq` exactly but in a single vectorised
    expression.

    Examples
    --------
    >>> fftfreq(8, d=0.001)
    array([    0.,   125.,   250.,   375.,  -500.,  -375.,  -250.,  -125.])

    See Also
    --------
    numpy.fft.fftfreq : NumPy’s reference implementation.
    numpy.fft.rfftfreq : Positive‑only frequencies for real‑input FFTs.
    numpy.fft.fftshift : Re‑order spectrum to `[−f..0..+f]` form.
    """
    k = np.arange(N)
    return ((k + N // 2) % N - N // 2) / (N * d)

In [ ]:
fftfreq(4)

In [ ]:
# verify this fftfreq function matches numpy/cupy 
assert np.abs(fftfreq(sample_size)-np.fft.fftfreq(random_sample.shape[-1])).max()<EPSILON

In [ ]:
my_fft_result_random = my_fft(random_sample)

# Perform IFFT to recover the original random sample
my_recovered_random_sample = np.array(my_ifft(fft_result_random))

In [ ]:
plot_random_sample_and_recovery(random_sample,my_recovered_random_sample)

In [ ]:
plot_frequencies(random_sample,my_fft_result_random)

In [ ]:
dft_result_random = my_fft(random_sample)

# Perform IFFT to recover the original random sample
dft_recovered_random_sample = np.array(my_ifft(fft_result_random))

In [ ]:
plot_random_sample_and_recovery(random_sample,dft_recovered_random_sample)

# Part Two

In [ ]:
inner_sims=2048
outer_sims=100000

In [ ]:
%%time
cp.random.seed(42)
rands=cp.random.randn(inner_sims,outer_sims)

In [ ]:
%%time
column_sums=rands.sum(axis=0)
column_sums.shape

In [ ]:
%%time
fft=cp.fft.fft(rands,axis=0)
fft.shape

In [ ]:
%%time
# first frequency always corresponds with sum of all values
assert np.abs(column_sums-fft[0]).max()<EPSILON

In [ ]:
mean_frequency_domain=fft.mean(axis=1)
mean_frequency_domain.shape

In [ ]:
%%time
# the mean of lowest frequency is the mean of column sums
assert np.abs(mean_frequency_domain[0]-column_sums.mean())<EPSILON

In [ ]:
column_sums.mean()/inner_sims

In [ ]:
%%time
ifft_mean_fft=cp.fft.ifft(mean_frequency_domain)

In [ ]:
%%time
# the ifft of the mean of the fft is the mean of the original random matrix
assert np.abs(ifft_mean_fft-rands.mean(axis=1)).max()<EPSILON

In [ ]:
fig=px.scatter(ifft_mean_fft.real.get(),opacity=0.3)
fig.update_traces(marker=dict(size=10))
fig

In [ ]:
px.histogram(ifft_mean_fft.real.get(),nbins=50)

In [ ]:
fig=px.violin(ifft_mean_fft.real.get(),points='all',box=True)
fig.update_traces(marker=dict(size=10,opacity=0.1))
fig

# Part Three - Visualization of DFT

In [ ]:
# Monte Carlo sample from standard normal distribution
np.random.seed(0)
N = 128
sample = np.random.normal(size=N)

# Compute the DFT of the sample
dft = np.fft.fft(sample)
real = dft.real
imag = dft.imag
magnitude = np.abs(dft)
phase = np.angle(dft)
idx = np.arange(N)

# Time series plot of the original sample
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=idx, y=sample, mode='lines+markers', name='Sample'))
fig1.update_layout(title='Original Sample (Time Series)', xaxis_title='Index', yaxis_title='Value')

# Plot real and imaginary parts of the DFT
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=idx, y=real, mode='lines+markers', name='Real part'))
fig2.add_trace(go.Scatter(x=idx, y=imag, mode='lines+markers', name='Imag part'))
fig2.update_layout(title='DFT: Real and Imaginary Parts', xaxis_title='Frequency Index', yaxis_title='DFT value')

# Plot magnitude and phase of the DFT (using subplots for clarity)
fig3 = make_subplots(rows=2, cols=1, shared_xaxes=True,
                     subplot_titles=('Magnitude of DFT', 'Phase of DFT'))
fig3.add_trace(go.Scatter(x=idx, y=magnitude, mode='lines+markers', name='Magnitude'), row=1, col=1)
fig3.add_trace(go.Scatter(x=idx, y=phase, mode='lines+markers', name='Phase'), row=2, col=1)
fig3.update_xaxes(title_text='Frequency Index', row=2, col=1)
fig3.update_yaxes(title_text='Magnitude', row=1, col=1)
fig3.update_yaxes(title_text='Phase (radians)', row=2, col=1)
fig3.update_layout(title='Magnitude and Phase of DFT', showlegend=False)

# 3D scatter plot: frequency index vs magnitude vs phase
fig4 = go.Figure()
fig4.add_trace(go.Scatter3d(x=idx, y=magnitude, z=phase, mode='markers',
                            marker=dict(size=4, color=magnitude, colorscale='Viridis', colorbar=dict(title='Magnitude'))))
fig4.update_layout(title='DFT: Frequency Index vs Magnitude vs Phase',
                   scene=dict(xaxis_title='Frequency Index', yaxis_title='Magnitude', zaxis_title='Phase (radians)'))

# Display the figures
fig1.show()
fig2.show()
fig3.show()
fig4.show()